In [1]:
import requests
import random
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'CoinCollector.settings')
django.setup()

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from base import models

# Parameters
base_url = 'https://api.numista.com/api/v3'
api_key = '71Vf19kBmiI6IBbo7yuqxtKArU1slD83YOhEVoH'
client_id = '378323'

In [2]:
# API call to get the list of issuers
response = requests.get(
  base_url + '/issuers',
  params={'lang': 'en'},
  headers={'Numista-API-Key': api_key})
issuers_response = response.json()

In [3]:
search_query = 'Q30'
filtered_issuers = [issuer for issuer in issuers_response['issuers'] if search_query == issuer['wikidata_id'] and issuer['parent'] == None]

for issuer in filtered_issuers:
    print(issuer)
    print()

{'code': 'united-states', 'name': 'United States', 'wikidata_id': 'Q30', 'parent': None}



In [4]:
issuers_codes = [
    'romania_section', #Q218
    'france_section', #Q142
    'belgium', #Q31
    'united-states', #Q30
]

issuers_currencies = {
    'romania_section' : 'RON',
    'france_section' : 'EUR',
    'belgium' : 'EUR',
    'united-states' : 'USD',
}


In [5]:
# API call to search types by issuer
def get_types(issuer_code, page=1):

  response = requests.get(
    base_url + '/types',
    params={'category': 'coin', 'issuer': issuer_code, 'count': 50, 'lang': 'en', 'page': page},
    headers={'Numista-API-Key': api_key})
  types_response = response.json()

  # Check response code (simplified)
  if response.status_code==401:
    raise Exception('The Numista API key is missing or incorrect') 
  return types_response['types']

In [6]:
def insert_coin(id, new_issuer):
    response = requests.get(
        base_url + '/types/' + str(id),
        params={'lang':'en'},
        headers={'Numista-API-Key': api_key})
    coin_details = response.json()
    new_coin = models.Coin( url=coin_details['url'],
                            title=coin_details['title'],
                            issuer=new_issuer,
                            minYear=coin_details['min_year'] if 'min_year' in coin_details else -1,
                            maxYear=coin_details['max_year'] if 'max_year' in coin_details else 1,
                            weight=coin_details['weight'] if 'weight' in coin_details else -1,
                            size=coin_details['size'] if 'size' in coin_details else -1,
    )
    new_coin.save()

    new_coinConverter = models.CoinConverter(coin=new_coin,
                                             amount = coin_details['value']['numeric_value'] if 'numeric_value' in coin_details['value'] else 0,
                                             currency = models.Currency.objects.get(code=issuers_currencies[new_issuer.code])
    )
    new_coinConverter.save()

In [8]:
for iss in issuers_codes:
    filtered_issuers = [issuer for issuer in issuers_response['issuers'] if issuer['code'] == iss]
    if len(filtered_issuers) == 1:
        new_issuer = models.Issuer(
            name=filtered_issuers[0]['name'],
            code=iss,
        )
        new_issuer.save()

        existing_currency = models.Currency.objects.filter(code=issuers_currencies[iss]).exists()
        if existing_currency:
            print("Currency already exists")
        else:
            new_currency = models.Currency(code=issuers_currencies[iss])
            new_currency.save()

    coins = get_types(iss)
    for type in coins:
        insert_coin(type['id'], new_issuer)

Currency already exists


In [21]:
existing_coin = models.Coin.objects.filter(title='Vaquette - Louis XIII (1st type)').exists()
print(existing_coin)

True
